<a href="https://colab.research.google.com/github/Jeffrey-Kidwell/ECE-535-Project/blob/main/VLM_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U torch torchvision torchaudio transformers accelerate sentencepiece pillow opencv-python scikit-learn soundfile
!pip -q install "pillow<12,>=8"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
import cv2
import numpy as np
import librosa
import moviepy.editor as mp
import tempfile

# -----------------------------
# 2️⃣ Load Qwen2-VL Model
# -----------------------------
MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

LABELS = ["asleep", "awake", "crying", "playing"]
print("Model loaded on:", device)

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

`torch_dtype` is deprecated! Use `dtype` instead!
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https:

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

Model loaded on: cuda


In [ ]:
# 3️⃣ Helper Functions
# -----------------------------
def _gen_reply_text(pil_image, prompt, max_new_tokens=16):
    messages = [
        {"role": "system", "content": [
            {"type": "text", "text": "You are a careful vision assistant for baby activity recognition."}
        ]},
        {"role": "user", "content": [
            {"type": "image", "image": pil_image},
            {"type": "text",  "text": prompt}
        ]}
    ]

    chat_text = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=False
    )
    inputs = processor(
        text=[chat_text], images=[pil_image], return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens)

    input_len = inputs["input_ids"].shape[1]
    gen_tokens = out[0][input_len:]
    reply = processor.tokenizer.decode(
        gen_tokens, skip_special_tokens=True
    ).strip()

    return reply


import re

def classify_activity(pil_image, audio_result=None):
    """Classify a frame + optional audio info."""

    # Force a single-word prediction
    forced_prompt = (
        "Classify the baby's current activity. "
        "Answer with EXACTLY ONE word from: asleep, awake, crying, playing."
    )

    resp = _gen_reply_text(pil_image, forced_prompt, max_new_tokens=8)
    m = re.search(r"\b(asleep|awake|crying|playing)\b", resp.lower())
    label = m.group(1) if m else "awake"

    # Natural-language visual summary
    summary = _gen_reply_text(
        pil_image,
        "In one short sentence, describe what the baby appears to be doing.",
        max_new_tokens=32
    ).splitlines()[0].strip()

    # Fusion logic (correct)
    if audio_result == "crying" or re.search(r"\bcry(ing)?\b", summary.lower()):
        label = "crying"

    return label, summary



def extract_audio_from_video(video_path, sr=16000):
    """Extract raw audio from video."""
    clip = mp.VideoFileClip(video_path)
    audio_path = tempfile.mktemp(suffix=".wav")
    clip.audio.write_audiofile(audio_path, fps=sr, verbose=False, logger=None)
    y, _ = librosa.load(audio_path, sr=sr)
    return y


def detect_crying(audio_waveform, sr=16000, threshold=0.02):
    """VERY simple crying detector."""
    energy = np.mean(np.abs(audio_waveform))
    return "crying" if energy > threshold else "not crying"


In [ ]:
# 4️⃣ File Upload
# -----------------------------
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]
print("File uploaded:", file_path)

# 5️⃣ Detect file type
# -----------------------------
VIDEO_EXTS = ('.mp4', '.mov', '.avi', '.webm')
is_video = file_path.lower().endswith(VIDEO_EXTS)

if is_video:

    # --- Audio Analysis ---
    audio_waveform = extract_audio_from_video(file_path)
    audio_result = detect_crying(audio_waveform)
    print("Audio detected crying?:", audio_result)

    # --- Video Frame Analysis --
    cap = cv2.VideoCapture(file_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = max(cap.get(cv2.CAP_PROP_FPS), 1)

    # Pick 3 representative frames: start, middle, end
    frame_indices = [
        0,
        total_frames // 2,
        max(total_frames - 1, 0)
    ]

    print("\n--- FAST Video Activity Monitoring (3 frames) ---")
    prev_label = None

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()

        if not ret:
            continue

        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        time_sec = idx / fps

        label, summary = classify_activity(pil_image, audio_result=audio_result)

        if label != prev_label:
            print(f"Time {time_sec:.2f}s: Label={label}, Summary={summary}")

            if label == "crying":
                print("🚨 Baby is crying! 🚨")
            elif label == "asleep":
                print("😴 Baby asleep")
            else:
                print("🙂 Baby active")

        prev_label = label

    cap.release()
    print("\n--- FAST video scan finished ---")


else:
    # --- Single Image ---
    pil_image = Image.open(file_path).convert("RGB")
    label, summary = classify_activity(pil_image, audio_result=None)

    print("\n--- Image Analysis ---")
    print(f"Label: {label}")
    print(f"Summary: {summary}")

    if label == "crying":
        print("🚨 Baby is crying! 🚨")
    elif label == "asleep":
        print("😴 Baby asleep")
    else:
        print("🙂 Baby active")


Saving ScreenRecording_11-26-2025 07-01-31_1.mov to ScreenRecording_11-26-2025 07-01-31_1 (1).mov
File uploaded: ScreenRecording_11-26-2025 07-01-31_1 (1).mov
Audio detected crying?: not crying

--- FAST Video Activity Monitoring (3 frames) ---
Time 0.00s: Label=playing, Summary=The baby is lying on a blanket with a pacifier in its mouth.
🙂 Baby active
Time 2.49s: Label=awake, Summary=The baby is lying on a blanket with a pacifier in its mouth.
🙂 Baby active

--- FAST video scan finished ---
